In [1]:
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import h5py




In [2]:
AU = 1.49597871 * (10 ** 11)

profile_files = ['../new_sim/sim10000_16.h5',
                '../new_sim/sim10000_17.h5',
                '../new_sim/sim10000_18.h5',
                '../new_sim/sim10000_19.h5',
                '../new_sim/sim10000_20.h5',
                '../new_sim/sim10000_21.h5']

ID_files = ['../ND_ID_10000_16.dat',
           '../ND_ID_10000_17.dat',
           '../ND_ID_10000_18.dat',
           '../ND_ID_10000_19.dat',
           '../ND_ID_10000_20.dat',
           '../ND_ID_10000_21.dat']

wf_files = ['../wf_16.dat',
           '../wf_17.dat',
           '../wf_18.dat',
           '../wf_19.dat',
           '../wf_20.dat',
           '../wf_21.dat']

all_data = []
# final_table = pd.DataFrame()

for profile_file, ID_file, wf_file in zip(profile_files, ID_files, wf_files):
    ID_data = np.loadtxt(ID_file)[:]
    wf_data = np.loadtxt(wf_file)[:]


    with h5py.File(profile_file, 'r') as f:
        lat = f['Scenario']['latitude'][:,0]#(10000,)
        lon = f['Scenario']['longitude'][:,0]#(10000,)
        sate_zenith = f['Scenario']['observation_zenith'][:,0]*np.pi/180#(10000,)
        sate_azimuth = f['Scenario']['observation_azimuth'][:,0]*np.pi/180#(10000,)
        solar_zenith  = f['Scenario']['solar_zenith'][:,0]*np.pi/180#(10000,)
        solar_azimuth = f['Scenario']['solar_azimuth'][:,0]*np.pi/180#(10000,)
        rela_v = f['Scenario']['relative_velocity'][:,0]#(10000,)
        ds = f['Scenario']['solar_distance'][:]/AU#(10000,)
        surf_p = f['Atmosphere']['surface_pressure'][:]#(10000,)

        p_level = f['Atmosphere']['pressure_levels'][:,:]#(10000,20)
        h2o = f['Atmosphere']['Absorber']['vmr'][:,1,:]#(10000,20)
        temp = f['Atmosphere']['temperature'][:,:]#(10000,20)
        co2_profile = f['Atmosphere']['Absorber']['vmr'][:,0,:]#(10000,20)

    xco2 = np.sum(co2_profile * wf_data, axis=1)#(10000,)
    # year_data = pd.DataFrame({'ID': ID_data, 'lat': lat, 'lon': lon, 'obs_z': sate_zenith, 'obs_a': sate_azimuth,\
    #                            'solar_z': solar_zenith, 'solar_a': solar_azimuth, 'velocity': rela_v, 'SE-d': ds, 'surf_P': surf_p,\
    #                            'P_profile': p_level,'h2o_profile': h2o,'T_profile': temp, 'co2_profile': co2_profile, 'wf': wf_data})
    # year_data['xco2'] = xco2#(10000,)

    year_data = np.column_stack((ID_data, lat, lon,sate_zenith,sate_azimuth,solar_zenith,solar_azimuth,rela_v,ds,surf_p,xco2,\
                                 p_level,h2o,temp,co2_profile,wf_data))

    all_data.append(year_data)

final_data = np.vstack(all_data)
# print(final_data.shape)
# column_names = ['ID', 'lat', 'lon','sate_zenith','sate_azimuth','solar_zenith','solar_azimuth','rela_v','ds','surf_p',\
#                 'p_level','h2o_profile','T_profile','co2_profile','wf', 'xco2'] 
final_table = pd.DataFrame(final_data)
    # final_table = pd.concat([final_table, year_data])

final_table.to_hdf('profile.h5', key='feature', mode='w')
